In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TinySpineNet(nn.Module):
    def __init__(self, num_classes=3):
        super(TinySpineNet, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.pool = nn.AdaptiveAvgPool2d((1,1))

        # <-- use a unified self.fc (Sequential) -->
        self.fc = nn.Sequential(
                    nn.Linear(256, 192),
                    nn.ReLU(),
                    nn.Dropout(0.35),
                    nn.Linear(192, 96),
                    nn.ReLU(),
                    nn.Dropout(0.2),
                    nn.Linear(96, num_classes)
                )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool(x)
        x = x.flatten(1)
        x = self.fc(x)
        return x


In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split

label_file = 'Dataset_Labels.xlsx'
df = pd.read_excel(label_file)
df.columns = ['Spine_Name', 'Spine_Label']

# Classify by category proportion to ensure fair comparison
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['Spine_Label']
)

train_df.to_csv("spine_train_split.csv", index=False)
test_df.to_csv("spine_test_split.csv", index=False)

print("Saved spine_train_split.csv and spine_test_split.csv")
print("Train size =", len(train_df), " Test size =", len(test_df))
print(train_df['Spine_Label'].value_counts())
print(test_df['Spine_Label'].value_counts())


Saved spine_train_split.csv and spine_test_split.csv
Train size = 364  Test size = 92
Spine_Label
Mushroom    230
Stubby       90
Thin         44
Name: count, dtype: int64
Spine_Label
Mushroom    58
Stubby      23
Thin        11
Name: count, dtype: int64


In [64]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import visdom
from PIL import Image
from torch.nn import CrossEntropyLoss
from torch import optim
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

# ==========================
# Hyperparameters
# ==========================
BATCH_SIZE = 32               # batch size
                # total training epochs
save_path = "./Spine_tinyCNN.pth"  # model save path

# ==========================
# Paths
# ==========================
img_dir = 'Dataset Binary'          # image folder
label_file = 'Dataset_Labels.xlsx'  # label file
train_df = pd.read_csv("spine_train_split.csv")
val_df   = pd.read_csv("spine_test_split.csv")


# ==========================
# Data Augmentation
# ==========================
data_transform = {
    "train": transforms.Compose([
        transforms.Resize((250, 250)),
        transforms.RandomRotation(10),                      # mild rotation
        transforms.RandomResizedCrop(250, scale=(0.9, 1.0)),# mild scale change
        transforms.RandomHorizontalFlip(p=0.5),             # useful if bilateral symmetry holds
        transforms.ToTensor(),
    ]),
    "val": transforms.Compose([
        transforms.Resize((250, 250)),
        transforms.ToTensor(),
    ])
}

# ==========================
# Custom Dataset
# ==========================
class BinarySpineDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.data = dataframe.reset_index(drop=True)
        self.root_dir = root_dir
        self.transform = transform

        # String → numeric label mapping
        self.label_map = {
            "Mushroom": 0,
            "Stubby": 1,
            "Thin": 2
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.root_dir, row['Spine_Name'])
        image = Image.open(img_path).convert("RGB")
        label = self.label_map[row['Spine_Label']]
        if self.transform:
            image = self.transform(image)
        return image, label

# ==========================
# Train / Val Split
# ==========================
train_df, val_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

train_dataset = BinarySpineDataset(train_df, img_dir, transform=data_transform["train"])
val_dataset   = BinarySpineDataset(val_df, img_dir, transform=data_transform["val"])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ==========================
# Device
# ==========================
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [67]:
import torch
import visdom
from PIL import Image
from torch.nn import CrossEntropyLoss
from torch import optim

BATCH_SIZE = 32
EPOCH = 100
save_path = "./Spine_tinyCNN.pth"

In [70]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import CrossEntropyLoss
import visdom

viz = visdom.Visdom(env="spine_exp")

# ====================================
# test function
# ====================================
def evalute(model, loader):
    correct = 0
    total = len(loader.dataset)
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            pred = out.argmax(dim=1)
            correct += (pred == y).float().sum().item()
    return correct / total


# ====================================
# create model
# ====================================
net = TinySpineNet(num_classes=3).to(device)

# If you wish to continue the training, please remove the following notes
checkpoint = torch.load(save_path)
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
best_acc = checkpoint['best_acc']
start_epoch = checkpoint['epoch']

# ====================================
optimizer = optim.Adam(net.parameters(), lr=0.0008)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, T_0=10, T_mult=2
)
loss_function = CrossEntropyLoss()

best_acc = 0.0
best_epoch = 0
start_epoch = 0
global_step = 0


# ====================================
# Training cycle
# ====================================
for epoch in range(start_epoch, EPOCH):
    running_loss = 0.0
    net.train()

    for step, (images, labels) in enumerate(train_loader, start=0):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        # train accuracy
        pred = outputs.argmax(dim=1)
        train_correct = (pred == labels).float().sum().item()
        train_acc = train_correct / labels.size(0)

        running_loss += loss.item()

        # progress bar
        rate = (step + 1) / len(train_loader)
        a = "*" * int(rate * 50)
        b = "." * int((1 - rate) * 50)
        print(
            "\repoch: {} train loss: {:^3.0f}%[{}->{}]{:.3f}  train_acc:{:.3f}".format(
                epoch + 1, int(rate * 100), a, b, loss, train_acc
            ),
            end="", flush=True
        )

        viz.line([train_acc], [global_step], win='train_acc', update='append')
        viz.line([loss.item()], [global_step], win='loss', update='append')

        global_step += 1

    # ===============================
    # test each epoch
    # ===============================
    test_acc = evalute(net, val_loader)
    print("  epoch{} test acc:{}".format(epoch + 1, test_acc))

    # visdom update
    viz.line([test_acc], [global_step], win='test_acc', update='append')

    # ===============================
    # save best model
    # ===============================
    if test_acc > best_acc:
        best_acc = test_acc
        best_epoch = epoch + 1

        torch.save({
            'epoch': best_epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_acc': best_acc
        }, save_path)

        print(f"🔥 Saved best model at epoch {best_epoch}, acc={best_acc:.4f}")

print("Finish !")
print("Best epoch:", best_epoch, " Best Acc:", best_acc)


Setting up a new session...
/var/folders/4n/_x3hbgpx0dz1zn2b0yt3w1_m0000gn/T/ipykernel_1445/1533478980.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torc

epoch: 1 train loss: 100%[**************************************************->]0.335  train_acc:0.917  epoch1 test acc:0.8043478260869565
🔥 Saved best model at epoch 1, acc=0.8043
epoch: 2 train loss: 100%[**************************************************->]0.254  train_acc:0.917  epoch2 test acc:0.8804347826086957
🔥 Saved best model at epoch 2, acc=0.8804
epoch: 3 train loss: 100%[**************************************************->]0.473  train_acc:0.833  epoch3 test acc:0.782608695652174
epoch: 4 train loss: 100%[**************************************************->]0.051  train_acc:1.000  epoch4 test acc:0.8260869565217391
epoch: 5 train loss: 100%[**************************************************->]0.379  train_acc:0.833  epoch5 test acc:0.9021739130434783
🔥 Saved best model at epoch 5, acc=0.9022
epoch: 6 train loss: 100%[**************************************************->]0.263  train_acc:0.833  epoch6 test acc:0.8260869565217391
epoch: 7 train loss: 100%[*********************